In [1]:
import pandas as pd
import numpy as np
import datetime

from scipy.stats import norm

import StoCalCodeDeck as stocal

## Problem 1

## Problem 2

## Problem 3

$$C_t = N(d_+)S_t - N(d_-)Ke^{r(T-t))}$$
$$d_{\pm} = \frac{\mathrm{ln}(\frac{S_t}{K}) + (r \pm \frac{\sigma^2}{2})(T-t))}{\sigma \sqrt{T}}$$

In [2]:
mu, sigma = stocal.walk_params(u=0.005,d=0.003,T=1,N=60)

s0 = 1000
k = 1025
r = 0.05
t = 1

In [3]:
def price_bsm(s0, k, r, t, sigma):
    d_plus = (np.log(s0/k) + (r + ((sigma ** 2) / 2)) * t) / (sigma * np.sqrt(t))
    d_less = (np.log(s0/k) + (r - ((sigma ** 2) / 2)) * t) / (sigma * np.sqrt(t))
    price = norm.cdf(d_plus) * s0 - norm.cdf(d_less) * k * np.exp(-r * t)
    return price

In [4]:
call_price = price_bsm(s0,k,r,t,sigma)
call_price

28.55095742703793

In [5]:
def put_call_parity(option_price, option_type, s0, k, r, t):
    if option_type == 'c':
        price = - s0 + np.exp(-r * t)*k + option_price
    elif option_type == 'p':
        price = s0 - np.exp(-r * t)*k + option_price 
    return price

In [6]:
put_price = put_call_parity(option_price=call_price,option_type='c',s0=1000,k=1025,r=0.05,t=1)
put_price

3.5611175402698336

In [7]:
put_call_parity(put_price,option_type='p',s0=1000,k=1025,r=0.05,t=1)

28.55095742703793

## Problem 4

In [11]:
'''Get ETF Data'''
etf, etf_data = stocal.asset_data(ticker = 'LQD', start_date='2017-01-01', end_date=datetime.datetime.today())
expiry, years_to_expiry = stocal.maturity(0.25, etf)
px_last = etf_data.iloc[-1,3]

'''Get Option Data'''
options = etf.option_chain(expiry)
calls = options.calls
puts = options.puts
itm_calls = calls[calls['strike'] < px_last - 0.5]
atm_calls = calls[(calls['strike'] < px_last+0.5) & (calls['strike'] > px_last-0.5)]
otm_calls = calls[calls['strike'] > px_last + 0.5]
atm_puts = puts[(puts['strike'] < px_last+0.5) & (puts['strike'] > px_last-0.5)]
atm_k = atm_calls['strike'].to_list()[0]
otm_k = 126             # Chosen arbitrarily
itm_k = 120             # Only ITM call left

'''Get Rates Data'''
rf = stocal.rates_data()

[*********************100%***********************]  1 of 1 completed


IndexError: list index out of range